In [3]:
##given performance in the previous page, where should I post next?
import pandas as pd
from pandas.stats.api import ols
from sklearn import datasets, linear_model
from sklearn.model_selection import train_test_split
from matplotlib import pyplot as plt
import math
%matplotlib inline
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.cross_validation import train_test_split
import math

from sklearn.cross_validation import cross_val_score, cross_val_predict
from sklearn import metrics

############ LOG SCALING - 
###PREDICTION TASK - with cross-promotion
###Cross-validation
train = pd.read_csv('jan_may_2017_links_cross_posting_info.csv')
##drop dups
# train = train.sort_values('ext_table1_firsthour_stats_date', ascending = False).drop_duplicates(subset=['buzz_post_buzz_id', 'ext_external_id'], keep = 'last')
all_pages = list(train.buzz_account_display_name.unique())

features = [
       'ext_table1_consumptions_by_type__link_clicks',
       'ext_table1_consumptions_by_type__other_clicks',
       'ext_table1_reactions_like_total',
       'ext_table1_stories_by_action_type__comment',
       'ext_table1_stories_by_action_type__like',
       'ext_table1_stories_by_action_type__share', 'ext_table1_engaged_fan',
       'ext_table1_fan_reach', 'ext_table1_impressions',
       'ext_table1_impressions_fan',
        ]

###convert to log-scale
# for feat in features:
#     train[feat] = np.log(train[feat]+1)

alt_inf = len(all_pages) + 1
train = train.replace(math.inf, alt_inf)

/Users/eshwarchandrasekharan/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [4]:
train.shape, train['buzz_post_buzz_id'].unique().shape

((37231, 113), (18793,))

In [5]:
###for each buzz_id, get the external ID which was posted FIRST!!!
train_grouped = train.groupby(['buzz_post_buzz_id']).agg({'buzz_post_created_at':'min'})

train_grouped = train_grouped.reset_index()

train_grouped = train_grouped.rename(columns={'buzz_post_created_at':'first_post'})

df = pd.merge(train, train_grouped, how='left', on=['buzz_post_buzz_id'])

df = df[df['buzz_post_created_at'] == df['first_post']]

In [12]:
tf = train

In [49]:
tel = train[train['buzz_post_buzz_id'] == 4456887].sort_values('buzz_post_created_at')

In [13]:
# def get_previous(X):
#     prev = -1
#     buzzid = train[train['buzz_external_id'] == X]['buzz_post_buzz_id'].values[0]    
#     created_at = train[train['buzz_external_id'] == X]['buzz_post_created_at'].values[0]    
#     tf = train[(train['buzz_post_buzz_id'] == buzzid)&(train['buzz_post_created_at'] < created_at)]
# #     print(len(tf))
#     if len(tf) < 1:
#         return -1
#     else:
#         prev = tf[-1:]['buzz_external_id'].values[0]
#         return prev

def get_previous(X):
#     print("Start")
    buzzid = train[train['buzz_external_id'] == X]['buzz_post_buzz_id'].values[0]
    cross_posts = list(train[train['buzz_post_buzz_id'] == buzzid].sort_values(['buzz_post_created_at'], ascending = True)['buzz_external_id']) 
    
#     print(cross_posts)
    index = cross_posts.index(X)
    if index == 0:
        return -1
#         return "-1"
    else:
        return cross_posts[index - 1]

tf['previous_external_id'] = tf['buzz_external_id'].apply(get_previous)

In [18]:
tf.to_csv('previous-page-info.csv', index = False)

In [19]:
####use performance in 1st hour on previous page to predict performance in 1st hour, 
####given a new page where it might get posted on!
tf.columns

Index(['buzz_external_id', 'buzz_post_parent_external_id',
       'buzz_post_created_at', 'buzz_account_display_name',
       'buzz_post_type_name', 'buzz_post_buzz_id', 'buzz_campaign_uri',
       'buzz_campaign_name', 'ext_table1_stats_date', 'ext_external_id',
       ...
       'BuzzFeed Sweaty', 'Tasty Junior', 'Oh Great, More Politics',
       'Einfach Tasty', 'BuzzFeed Steven L.', 'The Try Guys', 'Ohmygod Yaaa',
       'BuzzFeed Chloe', 'Nifty Science', 'previous_external_id'],
      dtype='object', length=114)

In [75]:
all_df = pd.read_csv('jan_may_2017_links_cross_posting_info.csv')
pre_df = pd.read_csv('previous-page-info.csv')

In [76]:
pre_df = pre_df[~(pre_df['previous_external_id'] == "-1")]
pre_df = pre_df[['buzz_external_id','previous_external_id']]
pre_df.columns = ['next_external_id', 'buzz_external_id']
pre_df = pd.merge(pre_df, all_df, on="buzz_external_id", how = "inner")

features = [
       'next_external_id', 
       'buzz_external_id', 
       'buzz_account_display_name', 
       'ext_table1_consumptions_by_type__link_clicks',
       'ext_table1_consumptions_by_type__other_clicks',
       'ext_table1_reactions_like_total',
       'ext_table1_stories_by_action_type__comment',
       'ext_table1_stories_by_action_type__like',
       'ext_table1_stories_by_action_type__share', 'ext_table1_engaged_fan',
       'ext_table1_fan_reach', 'ext_table1_impressions',
       'ext_table1_impressions_fan',
        ]

pre_df = pre_df[features]

post_df = all_df[['buzz_external_id', 'buzz_account_display_name', 'ext_table1_consumptions_by_type__link_clicks', 'ext_twodays_stories_by_action_type__share']]
post_df.columns = ['next_external_id', 'next_page', 'next_clicks', 'next_shares']

train_df = pd.merge(pre_df, post_df, on="next_external_id", how = "inner")

In [77]:
train_df[['ext_table1_consumptions_by_type__link_clicks', 'next_clicks']]

,ext_table1_consumptions_by_type__link_clicks,next_clicks
0,4757.0,3415.0
1,637.0,12213.0
2,80435.0,347.0
3,596.0,3345.0
4,33360.0,353.0
5,111.0,121.0
6,16669.0,1034.0
7,4102.0,1027.0
8,2149.0,860.0
9,3295.0,2384.0


In [81]:
train_features = [
#                    'buzz_account_display_name',
                   'ext_table1_consumptions_by_type__link_clicks',
                   'ext_table1_consumptions_by_type__other_clicks',
                   'ext_table1_reactions_like_total',
                   'ext_table1_stories_by_action_type__comment',
                   'ext_table1_stories_by_action_type__like',
                   'ext_table1_stories_by_action_type__share', 
                   'ext_table1_engaged_fan',
                   'ext_table1_fan_reach',
                   'ext_table1_impressions',
                   'ext_table1_impressions_fan', 
#                    'next_page',
                ]

In [82]:
print('SHARES: Adjusted R2')
res = ols(y = train_df['next_shares'], x = train_df[train_features])
print(res.r2_adj)
print('CLICKS: Adjusted R2')
res = ols(y = train_df['next_clicks'], x = train_df[train_features])
print(res.r2_adj)

SHARES: Adjusted R2
0.0498011118627
CLICKS: Adjusted R2
0.030265096955


/Users/eshwarchandrasekharan/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2881: FutureWarning: The pandas.stats.ols module is deprecated and will be removed in a future version. We refer to external packages like statsmodels, see some examples here: http://www.statsmodels.org/stable/regression.html
  exec(code_obj, self.user_global_ns, self.user_ns)
